In [0]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score,accuracy_score
import h5py
import tensorflow as tf

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
Xtrain = []
Ytrain = []
Xtest = []
Ytest = []

In [0]:
def wave_frontend(x, is_training):
    '''Function implementing the front-end proposed by Lee et al. 2017.
       Lee, et al. "Sample-level Deep Convolutional Neural Networks for Music
       Auto-tagging Using Raw Waveforms."
       arXiv preprint arXiv:1703.01789 (2017).
    - 'x': placeholder whith the input.
    - 'is_training': placeholder indicating weather it is training or test
    phase, for dropout or batch norm.
    '''
    initializer = tf.contrib.layers.variance_scaling_initializer()
    conv0 = tf.layers.conv1d(inputs=x,
                             filters=64,
                             kernel_size=3,
                             strides=3,
                             padding="valid",
                             activation=tf.nn.relu,
                             kernel_initializer=initializer)
    bn_conv0 = tf.layers.batch_normalization(conv0, training=is_training)

    conv1 = tf.layers.conv1d(inputs=bn_conv0,
                             filters=64,
                             kernel_size=3,
                             strides=1,
                             padding="valid",
                             activation=tf.nn.relu,
                             kernel_initializer=initializer)
    bn_conv1 = tf.layers.batch_normalization(conv1, training=is_training)
    pool_1 = tf.layers.max_pooling1d(bn_conv1, pool_size=3, strides=3)

    conv2 = tf.layers.conv1d(inputs=pool_1,
                             filters=64,
                             kernel_size=3,
                             strides=1,
                             padding="valid",
                             activation=tf.nn.relu,
                             kernel_initializer=initializer)
    bn_conv2 = tf.layers.batch_normalization(conv2, training=is_training)
    pool_2 = tf.layers.max_pooling1d(bn_conv2, pool_size=3, strides=3)

    conv3 = tf.layers.conv1d(inputs=pool_2,
                             filters=128,
                             kernel_size=3,
                             strides=1,
                             padding="valid",
                             activation=tf.nn.relu,
                             kernel_initializer=initializer)
    bn_conv3 = tf.layers.batch_normalization(conv3, training=is_training)
    pool_3 = tf.layers.max_pooling1d(bn_conv3, pool_size=3, strides=3)

    conv4 = tf.layers.conv1d(inputs=pool_3,
                             filters=128,
                             kernel_size=3,
                             strides=1,
                             padding="valid",
                             activation=tf.nn.relu,
                             kernel_initializer=initializer)
    bn_conv4 = tf.layers.batch_normalization(conv4, training=is_training)
    pool_4 = tf.layers.max_pooling1d(bn_conv4, pool_size=3, strides=3)

    conv5 = tf.layers.conv1d(inputs=pool_4,
                             filters=128,
                             kernel_size=3,
                             strides=1,
                             padding="valid",
                             activation=tf.nn.relu,
                             kernel_initializer=initializer)
    bn_conv5 = tf.layers.batch_normalization(conv5, training=is_training)
    pool_5 = tf.layers.max_pooling1d(bn_conv5, pool_size=3, strides=3)

    conv6 = tf.layers.conv1d(inputs=pool_5,
                             filters=256,
                             kernel_size=3,
                             strides=1,
                             padding="valid",
                             activation=tf.nn.relu,
                             kernel_initializer=initializer)
    bn_conv6 = tf.layers.batch_normalization(conv6, training=is_training)
    pool_6 = tf.layers.max_pooling1d(bn_conv6, pool_size=3, strides=3)

    return tf.expand_dims(pool_6, [3])
def spec_frontend(x, is_training, config, num_filt):
    '''Function implementing the proposed spectrogram front-end.
    - 'route_out': is the output of the front-end, and therefore the input of
        this function.
    - 'is_training': placeholder indicating weather it is training or test
        phase, for dropout or batch norm.
    - 'config': dictionary with some configurable parameters like: number of
        output units - config['numOutputNeurons'] or number of frequency bins
        of the spectrogram config['setup_params']['yInput']
    - 'num_filt': multiplicative factor that controls the number of filters
        for every filter shape.
    '''
    initializer = tf.contrib.layers.variance_scaling_initializer()
    y_input = config['setup_params']['yInput']
    input_layer = tf.expand_dims(x, 3)

    # padding only time domain for an efficient 'same' implementation
    # (since we pool throughout all frequency afterwards)
    input_pad_7 = tf.pad(input_layer,
                         [[0, 0], [3, 3], [0, 0], [0, 0]],
                         "CONSTANT")
    input_pad_3 = tf.pad(input_layer,
                         [[0, 0], [1, 1], [0, 0], [0, 0]],
                         "CONSTANT")

    # [TIMBRE] filter shape 1: 7x0.9f
    conv1 = tf.layers.conv2d(inputs=input_pad_7,
                             filters=num_filt,
                             kernel_size=[7, int(0.9 * y_input)],
                             padding="valid",
                             activation=tf.nn.relu,
                             kernel_initializer=initializer,reuse=tf.AUTO_REUSE)
    bn_conv1 = tf.layers.batch_normalization(conv1, training=is_training)
    pool1 = tf.layers.max_pooling2d(inputs=bn_conv1,
                                    pool_size=[1, bn_conv1.shape[2]],
                                    strides=[1, bn_conv1.shape[2]])
    p1 = tf.squeeze(pool1, [2])

    # [TIMBRE] filter shape 2: 3x0.9f
    conv2 = tf.layers.conv2d(inputs=input_pad_3,
                             filters=num_filt*2,
                             kernel_size=[3, int(0.9 * y_input)],
                             padding="valid",
                             activation=tf.nn.relu,
                             kernel_initializer=initializer,reuse=tf.AUTO_REUSE)
    bn_conv2 = tf.layers.batch_normalization(conv2, training=is_training)
    pool2 = tf.layers.max_pooling2d(inputs=bn_conv2,
                                    pool_size=[1, bn_conv2.shape[2]],
                                    strides=[1, bn_conv2.shape[2]])
    p2 = tf.squeeze(pool2, [2])

    # [TIMBRE] filter shape 3: 1x0.9f
    conv3 = tf.layers.conv2d(inputs=input_layer,
                             filters=num_filt*4,
                             kernel_size=[1, int(0.9 * y_input)],
                             padding="valid",
                             activation=tf.nn.relu,
                             kernel_initializer=initializer,reuse=tf.AUTO_REUSE)
    bn_conv3 = tf.layers.batch_normalization(conv3, training=is_training)
    pool3 = tf.layers.max_pooling2d(inputs=bn_conv3,
                                    pool_size=[1, bn_conv3.shape[2]],
                                    strides=[1, bn_conv3.shape[2]])
    p3 = tf.squeeze(pool3, [2])

    # [TIMBRE] filter shape 3: 7x0.4f
    conv4 = tf.layers.conv2d(inputs=input_pad_7,
                             filters=num_filt,
                             kernel_size=[7, int(0.4 * y_input)],
                             padding="valid",
                             activation=tf.nn.relu,
                             kernel_initializer=initializer,reuse=tf.AUTO_REUSE)
    bn_conv4 = tf.layers.batch_normalization(conv4, training=is_training)
    pool4 = tf.layers.max_pooling2d(inputs=bn_conv4,
                                    pool_size=[1, bn_conv4.shape[2]],
                                    strides=[1, bn_conv4.shape[2]])
    p4 = tf.squeeze(pool4, [2])

    # [TIMBRE] filter shape 5: 3x0.4f
    conv5 = tf.layers.conv2d(inputs=input_pad_3,
                             filters=num_filt * 2,
                             kernel_size=[3, int(0.4 * y_input)],
                             padding="valid",
                             activation=tf.nn.relu,
                             kernel_initializer=initializer,reuse=tf.AUTO_REUSE)
    bn_conv5 = tf.layers.batch_normalization(conv5, training=is_training)
    pool5 = tf.layers.max_pooling2d(inputs=bn_conv5,
                                    pool_size=[1, bn_conv5.shape[2]],
                                    strides=[1, bn_conv5.shape[2]])
    p5 = tf.squeeze(pool5, [2])

    # [TIMBRE] filter shape 6: 1x0.4f
    conv6 = tf.layers.conv2d(inputs=input_layer,
                             filters=num_filt * 4,
                             kernel_size=[1, int(0.4 * y_input)],
                             padding="valid",
                             activation=tf.nn.relu,
                             kernel_initializer=initializer,reuse=tf.AUTO_REUSE)
    bn_conv6 = tf.layers.batch_normalization(conv6, training=is_training)
    pool6 = tf.layers.max_pooling2d(inputs=bn_conv6,
                                    pool_size=[1, bn_conv6.shape[2]],
                                    strides=[1, bn_conv6.shape[2]])
    p6 = tf.squeeze(pool6, [2])

    # [TEMPORAL-FEATURES] - average pooling + filter shape 7: 165x1
    pool7 = tf.layers.average_pooling2d(inputs=input_layer,
                                        pool_size=[1, y_input],
                                        strides=[1, y_input])
    pool7_rs = tf.squeeze(pool7, [3])
    conv7 = tf.layers.conv1d(inputs=pool7_rs,
                             filters=num_filt,
                             kernel_size=165,
                             padding="same",
                             activation=tf.nn.relu,
                             kernel_initializer=initializer,reuse=tf.AUTO_REUSE)
    bn_conv7 = tf.layers.batch_normalization(conv7, training=is_training)

    # [TEMPORAL-FEATURES] - average pooling + filter shape 8: 128x1
    pool8 = tf.layers.average_pooling2d(inputs=input_layer,
                                        pool_size=[1, y_input],
                                        strides=[1, y_input])
    pool8_rs = tf.squeeze(pool8, [3])
    conv8 = tf.layers.conv1d(inputs=pool8_rs,
                             filters=num_filt*2,
                             kernel_size=128,
                             padding="same",
                             activation=tf.nn.relu,
                             kernel_initializer=initializer,reuse=tf.AUTO_REUSE)
    bn_conv8 = tf.layers.batch_normalization(conv8, training=is_training)

    # [TEMPORAL-FEATURES] - average pooling + filter shape 9: 64x1
    pool9 = tf.layers.average_pooling2d(inputs=input_layer,
                                        pool_size=[1, y_input],
                                        strides=[1, y_input])
    pool9_rs = tf.squeeze(pool9, [3])
    conv9 = tf.layers.conv1d(inputs=pool9_rs,
                             filters=num_filt*4,
                             kernel_size=64,
                             padding="same",
                             activation=tf.nn.relu,
                             kernel_initializer=initializer,reuse=tf.AUTO_REUSE)
    bn_conv9 = tf.layers.batch_normalization(conv9, training=is_training)

    # [TEMPORAL-FEATURES] - average pooling + filter shape 10: 32x1
    pool10 = tf.layers.average_pooling2d(inputs=input_layer,
                                         pool_size=[1, y_input],
                                         strides=[1, y_input])
    pool10_rs = tf.squeeze(pool10, [3])
    conv10 = tf.layers.conv1d(inputs=pool10_rs,
                              filters=num_filt*8,
                              kernel_size=32,
                              padding="same",
                              activation=tf.nn.relu,
                              kernel_initializer=initializer,reuse=tf.AUTO_REUSE)
    bn_conv10 = tf.layers.batch_normalization(conv10, training=is_training)

    # concatenate all feature maps
    pool = tf.concat([p1, p2, p3, p4, p5, p6, bn_conv7, bn_conv8, bn_conv9,
                      bn_conv10], 2)
    return tf.expand_dims(pool, 3)


def backend(route_out, is_training, config, num_units):
    '''Function implementing the proposed back-end.
    - 'route_out': is the output of the front-end, and therefore the input of
        this function.
    - 'is_training': placeholder indicating weather it is training or test
        phase, for dropout or batch norm.
    - 'config': dictionary with some configurable parameters like: number of
        output units - config['numOutputNeurons'] or number of frequency bins
        of the spectrogram config['setup_params']['yInput']
    - 'num_units': number of units/neurons of the output dense layer.
    '''
    initializer = tf.contrib.layers.variance_scaling_initializer()

    # conv layer 1 - adapting dimensions
    conv1 = tf.layers.conv2d(inputs=route_out,
                             filters=256,
                             kernel_size=[7, route_out.shape[2]],
                             padding="valid",
                             activation=tf.nn.relu,
                             name='1cnnOut',
                             kernel_initializer=initializer,reuse=tf.AUTO_REUSE)
    bn_conv1 = tf.layers.batch_normalization(conv1, training=is_training)
    bn_conv1_t = tf.transpose(bn_conv1, [0, 1, 3, 2])

    # conv layer 2 - residual connection
    bn_conv1_pad = tf.pad(bn_conv1_t,
                          [[0, 0], [3, 3], [0, 0], [0, 0]],
                          "CONSTANT")
    conv2 = tf.layers.conv2d(inputs=bn_conv1_pad,
                             filters=256,
                             kernel_size=[7, bn_conv1_pad.shape[2]],
                             padding="valid",
                             activation=tf.nn.relu,
                             name='2cnnOut',
                             kernel_initializer=initializer,reuse=tf.AUTO_REUSE)
    conv2_t = tf.transpose(conv2, [0, 1, 3, 2])
    bn_conv2 = tf.layers.batch_normalization(conv2_t, training=is_training)
    res_conv2 = tf.add(bn_conv2, bn_conv1_t)

    # temporal pooling
    pool1 = tf.layers.max_pooling2d(inputs=res_conv2, pool_size=[2, 1],
                                    strides=[2, 1], name='poolOut')

    # conv layer 3 - residual connection
    bn_conv4_pad = tf.pad(pool1,
                          [[0, 0], [3, 3], [0, 0], [0, 0]],
                          "CONSTANT")
    conv5 = tf.layers.conv2d(inputs=bn_conv4_pad,
                             filters=256,
                             kernel_size=[7, bn_conv4_pad.shape[2]],
                             padding="valid",
                             activation=tf.nn.relu,
                             name='3cnnOut',
                             kernel_initializer=initializer,reuse=tf.AUTO_REUSE)
    conv5_t = tf.transpose(conv5, [0, 1, 3, 2])
    bn_conv5 = tf.layers.batch_normalization(conv5_t, training=is_training)
    res_conv5 = tf.add(bn_conv5, pool1)

    # global pooling: max and average
    max_pool2 = tf.reduce_max(res_conv5, axis=1)
    avg_pool2, var_pool2 = tf.nn.moments(res_conv5, axes=[1])
    pool2 = tf.concat([max_pool2, avg_pool2], 2)
    flat_pool2 = tf.contrib.layers.flatten(pool2)

    # output - 1 dense layer with droupout
    flat_pool2_dropout = tf.layers.dropout(flat_pool2, rate=0.5,
                                           training=is_training)
    dense = tf.layers.dense(inputs=flat_pool2_dropout,
                            units=num_units,
                            activation=tf.nn.relu,
                            kernel_initializer=initializer,reuse=tf.AUTO_REUSE)
    bn_dense = tf.layers.batch_normalization(dense, training=is_training)
    dense_dropout = tf.layers.dropout(bn_dense, rate=0.5, training=is_training)
    return tf.layers.dense(inputs=dense_dropout,
                           activation=tf.sigmoid,
                           units=config['numOutputNeurons'],
                           kernel_initializer=initializer,reuse=tf.AUTO_REUSE)


def build_model(x, is_training, config):
    '''Function implementing an example of how to build a model with the
        functions above.
    - 'x': placeholder whith the input.
    - 'is_training': placeholder indicating weather it is training or test
        phase, for dropout or batch norm.
    - 'config': dictionary with some configurable parameters like: number of
        output units - config['numOutputNeurons'] or number of frequency bins
        of the spectrogram config['setup_params']['yInput']
    '''
    # The following line builds the model that achieved better results in our
    # experiments. It is based on a spectrogram front-end (num_filters=16)
    # with 500 output units in the dense layer.
    return backend(spec_frontend(x, is_training, config, 1), is_training,
                   config, 50)

In [0]:
tf.reset_default_graph()
os.chdir('gdrive/My Drive/MyMIR/final_Xtrain')

In [0]:
for i in range(0,12):
  if i<10:
    Xtrain = np.load('data'+str(i)+'.npy')
    os.chdir('../final_Ytrain')
    Ytrain = np.load('Y'+str(i)+'.npy')
    os.chdir('../final_Xtrain')
  elif i == 10:
    Xtrain = np.load('dataA.npy')
    os.chdir('../final_Ytrain')
    Ytrain = np.load('YA.npy')
    os.chdir('../final_Xtrain')
  elif i == 11:
    Xtrain = np.load('dataB.npy')
    os.chdir('../final_Ytrain')
    Ytrain = np.load('YB.npy')
    os.chdir('../final_Xtrain')
  else:
    Xtrain = np.load('dataC.npy')
    os.chdir('../final_Ytrain')
    Ytrain = np.load('YC.npy')
    os.chdir('../final_Xtrain')
  #Xtrain = Xtrain.reshape(Xtrain.shape[0],Xtrain.shape[1],Xtrain.shape[2],1)
  #Xval = Xval.reshape(Xval.shape[0],Xval.shape[1],Xval.shape[2],1)
  config = {'numOutputNeurons':50,'setup_params':{'yInput':10}}
  print(Ytrain.shape)
  Xtrain = np.float32(Xtrain)
  Ytrain = np.float32(Ytrain)
  Ypredict = build_model(Xtrain,1,config)
  with  tf.Session() as ses:
    ses.run(tf.initialize_all_variables())
    Ypredict = ses.run(Ypredict)#(Xtrain, Ytrain,batch_size=32,epochs=10,verbose=1,validation_split=0.1,validation_data = (Xval,Yval),shuffle=True)tf.device('/device:GPU:0') and
    print(roc_auc_score(Ytrain, Ypredict))
    #print(accuracy_score(Ytrain, Ypredict))

(1108, 50)


W0719 16:50:09.727943 140026153273216 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0719 16:50:11.989313 140026153273216 deprecation.py:323] From <ipython-input-4-d9fdeeea9ada>:112: conv2d (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
W0719 16:50:12.231842 140026153273216 deprecation.py:323] From <ipython-input-4-d9fdeeea9ada>:113: batch_normalization (from tensorflow.python.layers.normalization) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular

ValueError: ignored

In [0]:
tempx = [['dataD.npy','YD.npy'],['dataE.npy','YE.npy'],['dataF.npy','YF.npy']]
for tple in tempx:
  Xtestp,Ytestp = tple
  os.chdir('../Xtrain')
  Xtest = np.load(Xtestp)
  os.chdir('../Ytrain')
  Ytest = np.load(Ytestp)
  predicted = build_model(Xtest,0,config)
  with tf.Session() as ses:
    ses.run(Ypredict)
    print(roc_auc_score(Ytest, predicted))
    print(accuracy_score(Ytest, predicted))

In [0]:
os.listdir()

In [0]:
os.chdir('gdrive/My Drive/MyMIR/final_Xtrain')
data = []
Y = []
for i in range(0,9):
  if i<10:
    Xtrain = np.load('data'+str(i)+'.npy')
    os.chdir('../final_Ytrain')
    #Ytrain = np.load('Y'+str(i)+'.npy')
    os.chdir('../final_Xtrain')
  elif i == 10:
    Xtrain = np.load('dataA.npy')
    os.chdir('../final_Ytrain')
    #Ytrain = np.load('YA.npy')
    os.chdir('../final_Xtrain')
  elif i == 11:
    Xtrain = np.load('dataB.npy')
    os.chdir('../final_Ytrain')
    #Ytrain = np.load('YB.npy')
    os.chdir('../final_Xtrain')
  #Xtrain = Xtrain.reshape(Xtrain.shape[0],Xtrain.shape[1],Xtrain.shape[2],1)
  #Xval = Xval.reshape(Xval.shape[0],Xval.shape[1],Xval.shape[2],1)
  data.append(Xtrain)
  #Y.append(Ytrain)
  print('Xtrain'+str(i)+' Saved')
np.save('data',data)
#np.save('Y',Y)

NameError: ignored